In [1]:
import os
import torch
from peft import LoraConfig, get_peft_model, set_peft_model_state_dict
from transformers import (
    AutoModelForCausalLM,
)
from transformers import AutoTokenizer


In [2]:
model_name = "EleutherAI/pythia-2.8b"
#adapter_path = "./models/adapter_pythia-2.8b_ep50_bs4.pth"
adapter_path = "./models/adapter_pythia-2.8b_ep150_bs16.pth"
max_memory = {0: "22GIB", "cpu": "30GB"}

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto") #, max_memory=max_memory)

# small fix to get gpt generation working
model.config.pad_token_id = model.config.eos_token_id

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [4]:
peft_config = LoraConfig(
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=True,
    r=8, # better save and export the peft config used during training
    lora_alpha=16,
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)

/opt/conda/envs/dev/lib/python3.10/site-packages/peft/tuners/lora.py:173: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


In [5]:
adapter_weights = torch.load(adapter_path, map_location="cuda")
model = set_peft_model_state_dict(model, adapter_weights)

In [6]:
model = model.to("cuda")

def predict(x, num_samples=1):
    with torch.no_grad():
        with torch.autocast("cuda"):
            golden_sample = f"Write a modern and engaging job posting for the following basic qualifications: {x}\r\nResponse: \r\n"

            inputs = tokenizer(golden_sample, return_tensors="pt")
            outputs = model.generate(
                input_ids=inputs["input_ids"].to("cuda"),
                do_sample=True,
                temperature=0.9,
                # num_beams=5,
                max_length=1024,
                num_return_sequences=num_samples,
            )

            responses = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)
            
            for response in responses:
                print(os.linesep.join(response.split("\r\n")))
                print("-----------------------------------------------------------------")
        

In [10]:
requirements = """
- You will be part of Stepstone Emerging Technologies team
- You will redefine job search and job discovery experience for millions of users
- You will build MVPs and prototypes to test new ideas and technologies
"""

requirements = "\r\n".join(requirements.split(os.linesep))
predict(requirements)

Write a modern and engaging job posting for the following basic qualifications: 
- You will be part of Stepstone Emerging Technologies team
- You will redefine job search and job discovery experience for millions of users
- You will build MVPs and prototypes to test new ideas and technologies

Response: 
The ideal candidate will be excited and passionate about the opportunities and challenge of building innovative new products that help people find each other. Do you have vision and passion for how people find each other? Do you think about how to improve the user experience when browsing through millions of users? If yes, we want to talk to you. If you are inspired by the unlimited potential of the cloud, and the profound impact that new technology and architectures have on the way consumers use computers, we would like to talk to you. If you want to make disruptive improvements that dramatically change the way people work, we'd like to talk to you. If you are excited about architecti

In [12]:
requirements = """
- You want to be part of revolutionizing shoes
- You are an expert in Shoe AI
- You wear shoes all day long
- You love shoes
- You want to change the future with Amazon Shoes AI
- You are an expert in smart business shoes powered by Amazon Alexa
- You will work on the development of next generation smart shoes
- Proven track record developing next generation shoe applications
"""

requirements = "\r\n".join(requirements.split(os.linesep))
predict(requirements, num_samples=1)

Write a modern and engaging job posting for the following basic qualifications: 
- You want to be part of revolutionizing shoes
- You are an expert in Shoe AI
- You wear shoes all day long
- You love shoes
- You want to change the future with Amazon Shoes AI
- You are an expert in smart business shoes powered by Amazon Alexa
- You will work on the development of next generation smart shoes
- Proven track record developing next generation shoe applications

Response: 
All right, let's talk about the shoes. (Yes, that's plural. There are many pairs of shoes. And we're talking about shoes. Not clothes, nor books, not hardware nor software platforms. The shoes are on feet. Why would we talk about shoes? Because shoes are so important! When you step into a room, your shoes make a very loud squishing noise. They create a ton of friction and pressure as you walk. And when your shoes are broken or dirty, they make a very noticeable squeaking sound. Your shoes define you. And you can change tha